In [63]:

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.context import SparkContext
import pandas as pd

In [64]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

Chỉnh lại đường dẫn file cuả bạn

In [65]:
path = 'D:\\WORKSPACE\\DE\\study_de\\Big_Data\\Items Shared on 4-29-2023\\Dataset\\log_content\\20220401.json'

In [66]:
ds = spark.read.json(path)

In [67]:
ds.show()

+--------------------+-------+------+--------------------+-----+
|                 _id| _index|_score|             _source|_type|
+--------------------+-------+------+--------------------+-----+
|AX_momhia1FFivsGrn9o|history|     0|{KPLUS, HNH579912...|kplus|
|AX_momhca1FFivsGrnvg|history|     0|{KPLUS, HUFD40665...|kplus|
|AX_momhaa1FFivsGrnny|history|     0|{KPLUS, HNH572635...|kplus|
|AX_momhca1FFivsGrnvv|history|     0|{KPLUS, HND141717...|kplus|
|AX_momhia1FFivsGrn98|history|     0|{KPLUS, HNH743103...|kplus|
|AX_momg9a1FFivsGrnkS|history|     0|{KPLUS, HNH893773...|kplus|
|AX_momhca1FFivsGrnwA|history|     0|{KPLUS, HND083642...|kplus|
|AX_momhfa1FFivsGrn2u|history|     0|{KPLUS, DNFD74404...|kplus|
|AX_momhca1FFivsGrnwP|history|     0|{KPLUS, DTFD21200...|kplus|
|AX_momhca1FFivsGrnwU|history|     0|{KPLUS, LDFD05747...|kplus|
|AX_momhfa1FFivsGrn24|history|     0|{KPLUS, HNH063566...|kplus|
|AX_momhia1FFivsGrn-W|history|     0|{KPLUS, HNH866786...|kplus|
|AX_momhia1FFivsGrn-a|his

In [68]:
ds.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [69]:
ds.select('_source').printSchema()

root
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)



In [70]:
ds.select('_source.AppName', '_source.Contract', '_source.TotalDuration').show()

+-------+---------+-------------+
|AppName| Contract|TotalDuration|
+-------+---------+-------------+
|  KPLUS|HNH579912|          254|
|  KPLUS|HUFD40665|         1457|
|  KPLUS|HNH572635|         2318|
|  KPLUS|HND141717|         1452|
|  KPLUS|HNH743103|          251|
|  KPLUS|HNH893773|          924|
|  KPLUS|HND083642|         1444|
|  KPLUS|DNFD74404|          691|
|  KPLUS|DTFD21200|         1436|
|  KPLUS|LDFD05747|         1434|
|  KPLUS|HNH063566|          687|
|  KPLUS|HNH866786|          248|
|  KPLUS|NBAAA1128|          247|
|  KPLUS|HNH960439|          683|
|  KPLUS|HNJ035736|          246|
|  KPLUS|NTFD93673|         2288|
|  KPLUS|HNJ063267|         2282|
|  KPLUS|HNH790383|          906|
|  KPLUS|THFD12466|          242|
|  KPLUS|HNH566080|          242|
+-------+---------+-------------+
only showing top 20 rows



In [71]:
ds.select('_source.AppName').distinct().show()

+-------+
|AppName|
+-------+
|  KPLUS|
|  RELAX|
|  CHILD|
|CHANNEL|
|    VOD|
|   FIMS|
|  SPORT|
+-------+



Đặt thêm 1 cột là Category sẽ phân loại dựa theo App Name

In [72]:
ds = ds.withColumn('Category',
                when(
                     (col('_source.AppName') == 'KPLUS') | 
                     (col('_source.AppName') == 'RELAX'),
                     "Giải trí"
                    )
                .when(
                     (col('_source.AppName') == 'CHILD'),
                     "Trẻ em"
                    )
                .when(
                     (col('_source.AppName') == 'CHANNEL') |
                     (col('_source.AppName') == 'VOD'),
                     "Truyền hình"
                    )
                .when(
                     (col('_source.AppName') == 'FIMS'), 
                     "Phim ảnh"
                )
                .when(
                     (col('_source.AppName') == 'SPORT'),
                     "Thể thao"
                )
                .otherwise("Khác")
            )

In [73]:
ds.show()

+--------------------+-------+------+--------------------+-----+--------+
|                 _id| _index|_score|             _source|_type|Category|
+--------------------+-------+------+--------------------+-----+--------+
|AX_momhia1FFivsGrn9o|history|     0|{KPLUS, HNH579912...|kplus|Giải trí|
|AX_momhca1FFivsGrnvg|history|     0|{KPLUS, HUFD40665...|kplus|Giải trí|
|AX_momhaa1FFivsGrnny|history|     0|{KPLUS, HNH572635...|kplus|Giải trí|
|AX_momhca1FFivsGrnvv|history|     0|{KPLUS, HND141717...|kplus|Giải trí|
|AX_momhia1FFivsGrn98|history|     0|{KPLUS, HNH743103...|kplus|Giải trí|
|AX_momg9a1FFivsGrnkS|history|     0|{KPLUS, HNH893773...|kplus|Giải trí|
|AX_momhca1FFivsGrnwA|history|     0|{KPLUS, HND083642...|kplus|Giải trí|
|AX_momhfa1FFivsGrn2u|history|     0|{KPLUS, DNFD74404...|kplus|Giải trí|
|AX_momhca1FFivsGrnwP|history|     0|{KPLUS, DTFD21200...|kplus|Giải trí|
|AX_momhca1FFivsGrnwU|history|     0|{KPLUS, LDFD05747...|kplus|Giải trí|
|AX_momhfa1FFivsGrn24|history|     0|{

In thử ra 1 Contact là HND638705 xem nó hiện đang có dữ liệu như thế nào

In [74]:
ds.filter(ds['_source']['Contract'] == 'HND638705').select('_source.AppName', '_source.Contract', '_source.TotalDuration').show()

+-------+---------+-------------+
|AppName| Contract|TotalDuration|
+-------+---------+-------------+
|  KPLUS|HND638705|          607|
|CHANNEL|HND638705|           22|
|CHANNEL|HND638705|        10599|
|    VOD|HND638705|          287|
+-------+---------+-------------+



Tiến hành gom nhóm

In [75]:
grouped_data = ds.groupBy('_source.Contract', 'Category').agg(
    sum('_source.TotalDuration').alias('TotalDuration')
)

In [76]:

grouped_data.show()

+---------+-----------+-------------+
| Contract|   Category|TotalDuration|
+---------+-----------+-------------+
|HND638705|   Giải trí|          607|
|SGFD80170|Truyền hình|        46346|
|TBAAA1446|Truyền hình|        83545|
|HNH258681|Truyền hình|        65208|
|TQFD05495|Truyền hình|        19677|
|HYAAA2710|Truyền hình|        18854|
|TNFD04348|Truyền hình|        18837|
|LDFD11035|Truyền hình|        19409|
|QNAAA1707|Truyền hình|        47328|
|SGJ123154|Truyền hình|         8822|
|SLAAA0800|   Giải trí|          163|
|NTFD67846|   Giải trí|          162|
|HNH950968|Truyền hình|        82202|
|HND391656|   Giải trí|           98|
|SGH478512|Truyền hình|        86906|
|QNFD15420|Truyền hình|        18774|
|HNH450758|Truyền hình|        18744|
|HDFD09931|Truyền hình|        82707|
|SGJ106321|Truyền hình|        18741|
|SGD514548|Truyền hình|        55689|
+---------+-----------+-------------+
only showing top 20 rows



In ra dữ liệu và so với kết quả trước khi groupby thì thấy khớp

In [77]:
grouped_data.filter((grouped_data.Contract == 'HND638705') ).show()

+---------+-----------+-------------+
| Contract|   Category|TotalDuration|
+---------+-----------+-------------+
|HND638705|   Giải trí|          607|
|HND638705|Truyền hình|        10908|
+---------+-----------+-------------+



Tiến hành pivot

In [78]:
pivot_data = grouped_data.groupBy('Contract').pivot('Category').agg(sum('TotalDuration')).withColumnRenamed('Giải trí', 'TVDuration').withColumnRenamed('Phim ảnh', 'MovieDuration').withColumnRenamed('Trẻ em', 'ChildDuration').withColumnRenamed('Thể thao', 'SportDuration').withColumnRenamed('Truyền hình', 'RelaxDuration').na.fill(0)

In [79]:
pivot_data.show()

+---------+----------+-------------+-------------+-------------+-------------+
| Contract|TVDuration|MovieDuration|SportDuration|RelaxDuration|ChildDuration|
+---------+----------+-------------+-------------+-------------+-------------+
|SGJ041247|         0|            0|            0|           39|            0|
|BGFD09100|         0|            0|            0|        12525|            0|
|SGH671806|         0|            0|            0|           34|            0|
|HPD045982|         0|            0|            0|        25089|            0|
|KTFD09382|         0|            0|            0|         3868|            0|
|DNFD57491|         0|            0|            0|         6205|            0|
|SGAAC4404|         0|            0|            0|        44526|            0|
|DAFD62690|         0|            0|            0|        32864|            0|
|LDFD25252|         0|            0|            0|         7998|            0|
|HNH574564|         0|            0|            0|  

In [81]:
pivot_data.show()

+---------+----------+-------------+-------------+-------------+-------------+
| Contract|TVDuration|MovieDuration|SportDuration|RelaxDuration|ChildDuration|
+---------+----------+-------------+-------------+-------------+-------------+
|SGJ041247|         0|            0|            0|           39|            0|
|BGFD09100|         0|            0|            0|        12525|            0|
|SGH671806|         0|            0|            0|           34|            0|
|HPD045982|         0|            0|            0|        25089|            0|
|KTFD09382|         0|            0|            0|         3868|            0|
|DNFD57491|         0|            0|            0|         6205|            0|
|SGAAC4404|         0|            0|            0|        44526|            0|
|DAFD62690|         0|            0|            0|        32864|            0|
|LDFD25252|         0|            0|            0|         7998|            0|
|HNH574564|         0|            0|            0|  

In [82]:
pivot_data.filter((pivot_data.Contract == 'HND638705') ).show()

+---------+----------+-------------+-------------+-------------+-------------+
| Contract|TVDuration|MovieDuration|SportDuration|RelaxDuration|ChildDuration|
+---------+----------+-------------+-------------+-------------+-------------+
|HND638705|       607|            0|            0|        10908|            0|
+---------+----------+-------------+-------------+-------------+-------------+



In [ ]:
spark.stop()